In [1]:
import numpy as np
import pickle
import os
import tensorflow as tf

C:\Users\Chankoo\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
D = 5
K = 6
N = 10

In [3]:
ratings = [5,5,4,3,0]

In [4]:
W = {}

In [5]:
for k in range(K):
    W['w_'+str(k)] = np.random.randint(0,high = 5,size=(D,N) ,dtype='int16')

In [6]:
W2 = np.zeros((D,K,N),dtype='int16')

In [7]:
for k in range(K):
    for d in range(D): 
        W2[d][k] = W['w_'+str(k)][d]

In [8]:
W2.shape

(5, 6, 10)

In [284]:
W2

array([[[4, 2, 3, 1, 0, 0, 1, 3, 3, 3],
        [4, 2, 4, 1, 0, 1, 0, 3, 1, 4],
        [4, 2, 1, 1, 3, 0, 3, 2, 4, 2],
        [0, 3, 2, 2, 1, 2, 4, 3, 4, 2],
        [1, 1, 4, 3, 1, 4, 4, 2, 3, 4],
        [4, 4, 3, 4, 2, 0, 4, 2, 2, 2]],

       [[2, 1, 1, 1, 3, 0, 0, 0, 1, 0],
        [1, 3, 0, 0, 4, 2, 2, 4, 2, 1],
        [2, 4, 3, 4, 0, 3, 1, 4, 1, 1],
        [2, 4, 0, 2, 4, 1, 0, 4, 1, 3],
        [0, 2, 2, 1, 3, 1, 3, 4, 4, 0],
        [1, 0, 1, 3, 3, 2, 0, 3, 0, 1]],

       [[3, 1, 1, 2, 4, 1, 3, 2, 4, 0],
        [3, 1, 2, 3, 1, 0, 0, 3, 2, 1],
        [3, 2, 4, 0, 2, 2, 3, 1, 1, 0],
        [2, 2, 1, 3, 4, 3, 3, 1, 2, 2],
        [2, 1, 2, 3, 2, 3, 1, 2, 4, 1],
        [1, 4, 1, 4, 4, 3, 3, 2, 2, 2]],

       [[4, 1, 0, 3, 0, 1, 0, 0, 2, 1],
        [1, 1, 3, 4, 0, 1, 4, 3, 4, 4],
        [2, 3, 4, 4, 3, 4, 2, 2, 3, 1],
        [3, 1, 3, 4, 3, 0, 3, 4, 0, 3],
        [2, 0, 1, 2, 3, 0, 4, 4, 4, 0],
        [2, 1, 2, 2, 2, 2, 4, 2, 1, 0]],

       [[0, 4, 4, 1, 4, 4, 3, 1,

In [14]:
ratings = [5, 5, 4, 3, 0]

In [260]:
class LRR_Model:
    global W2
    W = W2
    SCORE_SQUARE = False
    T = 1000 # max iteration
    
    K = None # num of aspects
    D = None # num of documents
    N = None # num of words
    
    mu = None
    sigma = None
    sigma_inv = None
    delta_square = None
    beta = None
    
    aspect_weight = None # alpha
    alpha_hat = None 
    
    aspect_rating = None # S
    
    ratings = None
    
    _old_aspect_weight = None # old alpha
    _old_aspect_rating = None
    _old_beta = None
    
    def __init__(self,k,d,n,ratings:list):
        self.K = k
        self.D = d
        self.N = n
        self.ratings = np.array(ratings,dtype='float64')
        
        self.mu = np.array(np.random.rand(self.K) * 2.0 - np.ones(self.K) ,dtype='float64')
        self.sigma = np.array(np.identity(self.K),dtype='float64')
        self.sigma_inv = np.linalg.inv(self.sigma)
        
        self.beta = (2*np.random.rand(self.K, self.N)-1.0)/10
        self.delta_square = np.ones(1,dtype='float64')
        
        self.alpha_hat = np.random.normal(loc = self.mu, scale= self.sigma.diagonal(),size=(D,K) )
        
        self.aspect_weight = np.zeros((D,K))
        for d in range(D):
            self.aspect_weight[d] = np.exp(self.alpha_hat[d])/np.sum(np.exp(self.alpha_hat[d]))
            
        self._old_aspect_weight = np.zeros((D,K))
        self._old_aspect_rating = np.zeros((D,K))
        self._old_beta = np.zeros((K,N))
    
#     def calc_covariance(self,vct):
#         '''
#         :param vct:
#         :return:

#         double sum = 0, s;
# 		for(int i=0; i<m_k; i++){
# 			s = 0;
# 			for(int j=0; j<m_k; j++)
# 				s += vct[j] * m_sigma_inv[j][i];
# 			sum += s * vct[i];
# 		}
# 		return sum;

#         '''

#     def calc_det(self):
#         return np.linalg.det(self.sigma)

#     def calc_sigma_inv(self,scale):
#         self.sigma_inv = np.linalg.inv(self.sigma) * scale

    def save(self,file_name:str):
        print(os.path.join(os.getcwd(), os.path.join('model', file_name)))
        with open(os.path.join(os.getcwd(), os.path.join('model', file_name)), 'wb') as fp:
            pickle.dump(self,fp)

    @classmethod
    def load(cls,file_name:str):
        try:
            with open(os.path.join(os.getcwd(),os.path.join('model',file_name)),'rb') as fp:
                return pickle.load(fp)
        except Exception as e:
            print(e)
    
    def _set_aspect_rating(self):
        self._old_aspect_rating = self.aspect_rating
        tmp_aspect_rating = np.zeros((D,K))
        
        if self.SCORE_SQUARE==True:
            for d in range(D):
                tmp_aspect_rating[d] = 0.5*np.sum(np.multiply(self.beta,self.W[d]),axis=1)#to avoid negative rating 
            self.aspect_rating = tmp_aspect_rating
        else:
            for d in range(D):
                tmp_aspect_rating[d] = np.exp(np.sum(np.multiply(self.beta,self.W[d]),axis=1))#to avoid negative rating 
            self.aspect_rating = tmp_aspect_rating
            
    def _infer_aspect_weight(self):
        for d in range(D): # set the old aspect_weight
            self._old_aspect_weight[d] = np.exp(self.alpha_hat[d])/np.sum(np.exp(self.alpha_hat[d]))
            
        alpha_hat = tf.Variable(self.alpha_hat, name = 'aspect_weight_hat',dtype=tf.float64)
        
        mu = tf.placeholder_with_default(self.mu,shape=(K),name='mu')
        sigma_inv = tf.placeholder_with_default(self.sigma_inv,shape=(K,K),name='sigma_inv')
        delta_square = tf.placeholder_with_default(self.delta_square,shape=(1),name='delta_square')
        s = tf.placeholder_with_default(self.aspect_rating,shape=(D,K),name = 'aspect_rating')
        ratings = tf.placeholder_with_default(self.ratings,shape=(D),name = 'ratings')

#        # To get gradient
#         preceding_tmp = np.zeros((D,K),dtype='float32')
#         for d in range(D):
#             preceding_tmp[d] = (((self.ratings - np.matmul(self.alpha_hat_T[0],self.aspect_rating_T[0].T).diagonal())[d]*self.aspect_rating_T[0][d]) )

#         preceding = tf.placeholder_with_default(preceding_tmp,shape=(D,K),name='precending')
#         grad =  preceding/tf.square(delta) + tf.matmul((alpha_hat-mu),sigma_inv)

        loss = tf.square( ratings - tf.linalg.diag_part(tf.matmul(alpha_hat,tf.transpose(s)) ))/(2*tf.square(delta_square)) + 0.5*tf.matmul(tf.matmul(tf.subtract(alpha_hat,mu), sigma_inv),tf.transpose(tf.subtract(alpha_hat,mu)))

        optimizer = tf.train.GradientDescentOptimizer(0.01)
        train = optimizer.minimize(loss)
        
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())

            for step in range(201):
                res = sess.run([alpha_hat,train])
                if step % 20 == 0:
                    print(step,':')
                    print('loss:',sess.run(loss),'\n') 
                    print('alpha_hat:',sess.run(alpha_hat),'\n')
        
        self.alpha_hat = res[0] # set a new alpha_hat
        for d in range(D): # set a new aspect_weight
            self.aspect_weight[d] = np.exp(self.alpha_hat[d])/np.sum(np.exp(self.alpha_hat[d]))

    
    def e_step():
        self._set_aspect_rating() # step 1: estimate aspect rating & update
        self._infer_aspect_weight() # step 2: infer aspect weight & update
    
    def _infer_beta(self):
        self._old_beta = self.beta
        
        # 텐서플로까지(beta_rep이 Variable) 한번에 코드짤수있는 방안??
        W_beta = np.zeros(D,K,N) # multiply of W & beta
        for d in range(D):
            W_beta = np.multiply(self.beta,self.W[d])
        
        
        beta = tf.Variable(np.tile(self.beta,self.N).reshape(self.K,self.N), name = 'beta_rep',dtype=tf.float64)
        
        alpha = tf.placeholder_with_default(self.aspect_weight,shape=(D,K),name='aspect_weight')
        # W(K,N) 인메모리??
        delta_square = tf.placeholder_with_default(self.delta_square,shape=(1),name='delta_square')
        sigma_inv = tf.placeholder_with_default(self.sigma_inv,shape=(K,K),name='sigma_inv')
        ratings = tf.placeholder_with_default(self.ratings,shape=(D),name = 'ratings')
        
        W = tf.placeholder_with_default(self.W,shape=(D,K,N),name='W')
        
        loss = tf.reduce_sum(
                tf.math.divide(
                              tf.square(tf.subtract(ratings, tf.reduce_sum(tf.math.multiply(
                                                                                  tf.reduce_sum(
                                                                                        tf.math.multiply(beta,
                                                                                                               W)
                                                                                                        ,axis=1) 
                                                                                                ,axis=1)
                                                                                            ,alpha )
                                                                          )
                                                   )
                                       )
                ,delta_square) 
                            )
    
        optimizer = tf.train.GradientDescentOptimizer(0.01)
        train = optimizer.minimize(loss)
        
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())

            for step in range(201):
                res = sess.run([beta,train])
                if step % 20 == 0:
                    print(step,':')
                    print('loss:',sess.run(loss),'\n') 
                    print('alpha_hat:',sess.run(beta),'\n')
        
            self.beta = res[0] # set a new beta
            
    
    
    def m_step(self): # m-step can only be applied to training samples
        # updateSigma = false; // shall we update Sigma?
        # Step 0: initialize the statistics
        # Step 1: ML for \mu
        self.mu = (np.sum(self.mu,axis=0))/self.D
        # Step 2: ML for \sigma
        #######################
        
        # Step 3: ML for \delta
        self.delta_square = np.sum(np.square( self.ratings - np.diagonal(np.matmul(self.aspect_weight, np.transpose(self.aspect_rating)) ))) / self.D
        
        #Step 4: ML for \beta
        self._infer_beta()
        
        
        
    
    def em_est(converge:float):
#         e_step()
#         m_step()
        pass

In [261]:
model = LRR_Model(6,5,10,ratings)

In [262]:
model.save('test.model')

C:\Users\Chankoo\Desktop\GitHub\BOAZ-projects\airbnb-NLP\model\test.model


In [263]:
model = LRR_Model.load('test.model')

In [264]:
model._set_aspect_rating()

In [265]:
model._old_aspect_rating

In [266]:
model.aspect_rating

array([[1.36106182, 0.99872305, 0.74516358, 1.25071528, 1.42804492,
        0.53300072],
       [1.02209736, 1.20229705, 0.48964648, 1.07968697, 1.77028983,
        0.67562646],
       [1.45839343, 0.97014964, 0.53784751, 1.28753277, 1.17611385,
        0.62682705],
       [1.07398682, 1.30253731, 0.41614316, 0.95925696, 1.31965487,
        0.50620148],
       [1.93525733, 0.85452814, 0.68011651, 1.4087055 , 1.54593813,
        0.47987615]])

In [267]:
model.aspect_rating

array([[1.36106182, 0.99872305, 0.74516358, 1.25071528, 1.42804492,
        0.53300072],
       [1.02209736, 1.20229705, 0.48964648, 1.07968697, 1.77028983,
        0.67562646],
       [1.45839343, 0.97014964, 0.53784751, 1.28753277, 1.17611385,
        0.62682705],
       [1.07398682, 1.30253731, 0.41614316, 0.95925696, 1.31965487,
        0.50620148],
       [1.93525733, 0.85452814, 0.68011651, 1.4087055 , 1.54593813,
        0.47987615]])

In [268]:
model._old_aspect_weight

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [269]:
model._infer_aspect_weight() 

0 :
loss: [[ 6.67540185  0.33186554 17.10572114  3.0387028   5.17478016]
 [-0.78297208  2.90871865 17.81861768  3.56717242  3.48067579]
 [-0.56327276  1.26446139 20.15256008  2.45999157  1.77437776]
 [ 0.73678194  2.38008916 17.82706461  4.64650374  4.25242305]
 [ 2.69148048  2.11221372 16.96007197  4.07104423  7.42305305]] 

alpha_hat: [[ 2.18948722 -1.23016182  1.40811658  0.04839526  1.39406684 -0.7736269 ]
 [ 0.75857383  0.60420913 -1.0294542  -0.64549924  0.96591075  2.06388357]
 [ 0.74330791  0.20525221 -1.83325883 -1.23205516 -0.54668929 -0.06188477]
 [ 0.80443046 -0.33222987 -0.15760771 -1.84938611  1.0045901   1.70899648]
 [ 0.70187306 -2.12693348  0.12502977  0.17643918  0.95268715  1.9813087 ]] 

20 :
loss: [[ 6.42327835 -0.63930911  4.43095631  1.26644962  2.13853885]
 [-0.90568666  2.16546415  4.95847844  1.95473634  0.73598803]
 [-0.18056052  0.61333916  6.32384217  0.58584839 -0.62511692]
 [ 0.54694322  1.50160749  4.47785882  2.79627459  1.5098177 ]
 [ 1.90057921  0.764

NameError: name 'alpha' is not defined

In [ ]:
model.aspect_weight

In [270]:
model.mu

array([ 0.40823205,  0.4831515 , -0.17405096, -0.47696896,  0.09275198,
        0.79349162])

In [271]:
model.delta_square

array([1.])

In [272]:
model.m_step()

In [273]:
model.mu

0.2253214476301908

In [274]:
model.delta_square

8.821404045634148